# Baseline solution

## Delete
The concept for this approach draws inspiration from how mass media handles offensive language. For instance, in TV programs and articles, swear words are often obscured or substituted with asterisks (`*`). Following a similar principle, goal of this baseline is to remove offensive words from the source text. 

To perform the baseline solution, I have created a dictionary containing the most frequently used swear words from the obscenity list taken from [here](https://github.com/surge-ai/profanity/tree/main). Using nltk library, I cleaned the reference sentences. Any words that match the words in that dictionary of swear words are then removed filtering out toxic language.


In [1]:
%cd ..
from tqdm import tqdm
import pandas as pd
import requests
import io
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

/Users/leiluk1/innopolis/text-detoxification


/Users/leiluk1/innopolis/text-detoxification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[nltk_data] Downloading package punkt to /Users/leiluk1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# Load the dictionary of profanity words
def download_csv_and_save_column(url, file_path):
    # Download the CSV file using the link
    response = requests.get(url)
    
    # Load the CSV data into a DataFrame
    df = pd.read_csv(io.StringIO(response.text))
    
    # Save the dictionary to a text file (+ add some toxic words that not present in dictionary)
    profanity_vocab = set(df['canonical_form_1'].to_list() + ['idiot', 'stupid', 'fucking', 'nigga'])
    with open(file_path, 'w') as file:
        for word in profanity_vocab:
            file.write(word + '\n')
    

vocab_url = 'https://github.com/surge-ai/profanity/raw/main/profanity_en.csv'
vocab_path = './data/external/profanity_vocab.txt'
download_csv_and_save_column(vocab_url, vocab_path)

In [3]:
# Create the dictionary of toxic words 
with open(vocab_path) as vocab_file:
        dictionary = [line.strip().split()[0] for line in vocab_file.readlines()]

dictionary[:6]

['grope', 'creampie', 'jizz', 'snatch', 'queer', 'kike']

Since for this baseline training process is not required, I will perform it only on test set and then save the results for further evaluation.

In [4]:
df = pd.read_csv('./data/interim/test.csv')

source = df['reference'].to_list()
target = df['detox_reference'].to_list()

In [5]:
def delete_toxic_words(texts, bad_words):
    """
    Deletes all toxic words from a pre-defined vocab in a list of source texts,
    and writes the resulting texts to an array.

    Args:
        texts: A list of strings containing the source texts to be cleaned.
        bad_words: The obscene words dictionary.

    Returns:
        A list of cleaned texts with toxic words removed.
    """

    results = []

    for text in tqdm(texts, desc="Deleting toxic words..."):
        # Tokenize the text into words
        text = word_tokenize(text)
        
        # Remove all toxic words from the text
        clean_text = ' '.join(
            [word for word in text if word.lower() not in bad_words]
        )
        
        # Remove extra spaces before punctuation
        clean_text = re.sub(r'\s+([^\w\s])', r'\1', clean_text)
        
        results.append(clean_text)

    return results

In [6]:
result = delete_toxic_words(source, bad_words=dictionary)

Deleting toxic words...: 100%|██████████| 5000/5000 [00:00<00:00, 12610.09it/s]


In [7]:
# Save the results to a CSV file
df['baseline_result'] = result

resulting_dataset_path = './data/interim/baseline_results.csv'
df.to_csv(resulting_dataset_path, index=False)

## Some examples

In [8]:
examples = ["What a stupid joke.", "Fucking damn joke!"]
baseline_result = delete_toxic_words(examples, bad_words=dictionary)

Deleting toxic words...: 100%|██████████| 2/2 [00:00<00:00, 11966.63it/s]


In [9]:
for i, example in enumerate(examples):
    print('Source:', example)
    print('Baseline result:', baseline_result[i])
    print()

Source: What a stupid joke.
Baseline result: What a joke.

Source: Fucking damn joke!
Baseline result: joke!



As can be seen, the bad words from the source sentences were removed. However, this approach is not the perfect one as it may result in the loss of contextual information within the original toxic sentence.